In [11]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import winsound
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report

from keras.utils import np_utils

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy


print(f" Found and Using {len(tf.config.experimental.list_physical_devices('GPU'))} GPU")

 Found and Using 1 GPU


In [12]:
def scale_x_encode_y(df, target_col):

    x = df.loc[:, df.columns != target_col]
    y = df[target_col]

    x_scaled = StandardScaler().fit_transform(x.values)
    
    
    encoder = LabelEncoder()
    encoder.fit(y)
    y_encoded = encoder.transform(y)
    y_encoded = np_utils.to_categorical(y_encoded)
    
    y_map = dict(zip(encoder.transform(encoder.classes_),encoder.classes_))
 
    return(x_scaled, y_encoded, y_map)

In [13]:
df = pd.read_csv('data/rolled_df.csv')
#df = pd.read_csv('data/IRIS.csv')

print(len(df))
df.head(10)

24821


,month,day,hour,min,0,1,2,3,4,5,6,7,8,9,10,11,12,13,direction
0,1,1,20,47,0.00009,0.00000,0.00021,0.00027,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,decrease
1,1,1,21,2,0.00000,0.00021,0.00027,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,increase
2,1,1,21,17,0.00021,0.00027,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,same
3,1,1,21,32,0.00027,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,decrease
4,1,1,21,47,0.00002,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,decrease
5,1,1,22,2,0.00005,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,increase
6,1,1,22,17,-0.00026,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,0.00025,same
7,1,1,22,32,-0.00050,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,0.00025,0.00006,increase
8,1,1,22,47,0.00034,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,0.00025,0.00006,0.00016,decrease
9,1,1,23,2,-0.00056,-0.00004,-0.00009,-0.00019,-0.00011,-0.00023,0.00029,-0.00001,-0.00034,-0.00059,0.00025,0.00006,0.00016,-0.00021,decrease


In [14]:
x_scaled, y_encoded, y_map = scale_x_encode_y(df, 'direction')

In [15]:
train_x, valid_x, train_y, valid_y = train_test_split(x_scaled, y_encoded,train_size = 0.8,random_state = 1)

In [16]:
model = Sequential([
    Dense(units = 16, activation='relu', input_shape=[len(train_x[0])]),
    Dense(units = 32, activation='relu'),
    Dense(units = 3, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 16)                304       
_________________________________________________________________
dense_4 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 99        
Total params: 947
Trainable params: 947
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),    
    metrics=['accuracy'])

In [20]:
epoch_val         = 10
batch_size_val    = 10

verbose_val       = 2
workers_val       = -1

In [21]:
model.fit(x=x_scaled, 
          y=y_encoded, 
          epochs=epoch_val,
          batch_size = batch_size_val,
          validation_data = (valid_x, valid_y),
          workers= workers_val,
         verbose=verbose_val)

Epoch 1/10
2483/2483 - 5s - loss: 1.0503 - accuracy: 0.4484 - val_loss: 1.0479 - val_accuracy: 0.4550
Epoch 2/10
2483/2483 - 5s - loss: 1.0470 - accuracy: 0.4535 - val_loss: 1.0454 - val_accuracy: 0.4540
Epoch 3/10
2483/2483 - 5s - loss: 1.0435 - accuracy: 0.4582 - val_loss: 1.0432 - val_accuracy: 0.4624
Epoch 4/10
2483/2483 - 5s - loss: 1.0423 - accuracy: 0.4578 - val_loss: 1.0405 - val_accuracy: 0.4566
Epoch 5/10
2483/2483 - 5s - loss: 1.0402 - accuracy: 0.4592 - val_loss: 1.0381 - val_accuracy: 0.4679
Epoch 6/10
2483/2483 - 5s - loss: 1.0394 - accuracy: 0.4619 - val_loss: 1.0363 - val_accuracy: 0.4661
Epoch 7/10
2483/2483 - 5s - loss: 1.0380 - accuracy: 0.4630 - val_loss: 1.0368 - val_accuracy: 0.4620
Epoch 8/10
2483/2483 - 5s - loss: 1.0368 - accuracy: 0.4636 - val_loss: 1.0355 - val_accuracy: 0.4655
Epoch 9/10
2483/2483 - 5s - loss: 1.0362 - accuracy: 0.4622 - val_loss: 1.0333 - val_accuracy: 0.4657
Epoch 10/10
2483/2483 - 5s - loss: 1.0351 - accuracy: 0.4642 - val_loss: 1.0336 - 

In [22]:
predictions = model.predict(valid_x)
rounded_predictions = np.argmax(predictions, axis = -1)
rounded_valid_y = np.argmax(valid_y, axis = -1)

print(classification_report(rounded_valid_y, rounded_predictions, target_names = y_map.values()))

              precision    recall  f1-score   support

    decrease       0.44      0.36      0.40      1581
    increase       0.42      0.40      0.41      1594
        same       0.52      0.63      0.57      1790

    accuracy                           0.47      4965
   macro avg       0.46      0.46      0.46      4965
weighted avg       0.46      0.47      0.46      4965

